In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

: 

In [1]:
import pandas as pd
path = 'C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/dataset/'
df = pd.read_csv(path+'기술과학분야_train_set.csv',index_col=0)
df_val = pd.read_csv(path+'기술과학분야_valid_set.csv',index_col=0)

In [2]:
df = df.drop(index=df[df['domain'] == '기후'].index)
df = df.reset_index(drop=True)

In [3]:

train = pd.concat([df.loc[df['domain']=='기술과학'].sample(150000),df.loc[df['domain']=='세계'].sample(150000),df.loc[df['domain']=='정치'].sample(150000),df.loc[df['domain']=='경제'].sample(150000)]).sample(600000).reset_index(drop=True)
valid = pd.concat([df.loc[df['domain']=='기술과학'].sample(5000),df.loc[df['domain']=='세계'].sample(5000),df.loc[df['domain']=='정치'].sample(5000),df.loc[df['domain']=='경제'].sample(5000)]).sample(20000).reset_index(drop=True)

train['labels'] = train['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])
valid['labels'] = valid['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])

# 모델링

In [4]:
! pip install transformers

In [5]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
# 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

c:\Users\withus\.conda\envs\ML\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSeq

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
class CustomDataset(Dataset):

    def __init__(self,text, label,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = label
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        text = str(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float),
            }
    


training_set = CustomDataset(train['ko'],train['labels'], tokenizer, 128)
testing_set = CustomDataset(valid['ko'], valid['labels'],tokenizer, 128)

train_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [7]:
loss_fn = nn.CrossEntropyLoss() 

LEARNING_RATE = 5e-5
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)


train_steps = len(training_loader.dataset) // train_params['batch_size']
val_steps = len(testing_loader.dataset) // test_params['batch_size']


In [ ]:
from tqdm import tqdm
epochs= 20
PATH = "C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/model_output/"

for epoch in (range(epochs)):
    model.train()
    num_correct = 0
    trainning_loss = 0
    for i, data in enumerate(tqdm(training_loader)):
        
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        label = data['labels'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        
        output = model(ids, mask, token_type_ids)
        print(output)
        loss = loss_fn(output[0], label)
        trainning_loss += loss
        loss.backward()
        optimizer.step()
        if (i+1) % 1000 == 0:
          print("Training Losses: {}".format(trainning_loss/i+1))
          print("Trainong Accuracy:{}".format(num_correct/i+1))
    
        pred = output[0].argmax(dim=1)
        num_correct += torch.eq(pred,label).sum().float().item()
        


    print('EPOCH ', epoch+1)
    print("Training Losses: {}".format(trainning_loss/len(training_loader.dataset)))
    print("Trainong Accuracy:{}".format(num_correct/len(training_loader.dataset)))
    
    
    
    if epoch % 5 == 0 or epoch == epochs-1:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': trainning_loss,
                    }, PATH+f"model600000data{epoch}.pt")
  

: 

In [9]:
checkpoint = torch.load('C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/model_output/model600000data4.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [10]:
model.eval()
num_correct = 0

for i, data in enumerate(tqdm(testing_loader)):

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      label = data['labels'].to(device, dtype = torch.long)


      optimizer.zero_grad()

      output = model(ids, mask, token_type_ids)
      loss = loss_fn(output[0], label)
      optimizer.step()

      pred = output[0].argmax(dim=1)
      num_correct += torch.eq(pred, label).sum().float().item()


print('EPOCH ', epoch+1)
print("validation Losses: {}".format(loss))
print("validation Accuracy:{}".format(num_correct/len(testing_loader.dataset)))


  0%|          | 0/625 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\withus\.conda\envs\ML\lib\site-packages\transformers\tokenization_utils_base.py:2302: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 625/625 [00:34<00:00, 18.27it/s]

EPOCH  5
validation Losses: 0.020893540233373642
validation Accuracy:0.97315


# keyphrase classification(키워드 뽑은거 돌려 보는 부분 부분)


In [11]:
import json

file_path = "C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase/임대차3법/KeyBERT/keywords.json"

with open(file_path, 'r') as file:
    keywords = json.load(file)
    

In [12]:
category = ['사회', '경제', '기술과학','정치']
keyword_params = {'batch_size': 5,
                'shuffle': True,
                'num_workers': 0
                }
for keyword in keywords:
  keyword_5 = list(keyword.values())[0]
  keyword_set = CustomDataset(keyword_5,[-1]*len(keyword_5), tokenizer, 128)
  keyword_loader = DataLoader(keyword_set, **keyword_params)
  for i, data in enumerate((keyword_loader)):

    ids = data['input_ids'].to(device, dtype = torch.long)
    mask = data['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    label = data['labels'].to(device, dtype = torch.long)

    output = model(ids, mask, token_type_ids)
    
    pred = pd.Series(output[0].argmax(dim=1).cpu().numpy()).replace([0,1,2,3],category)
    print([(keyword_5[i],pred[i]) for i in range(5)])

[('부동산 개편', '경제'), ('전월세 시장', '사회'), ('안정화 부총리', '경제'), ('부총리 임대차', '경제'), ('임대주택 개정', '경제')]
[('수요 월세', '사회'), ('상승 임대차', '경제'), ('부동산 시장', '경제'), ('셋값 상승', '경제'), ('강남 셋값', '경제')]
[('앵커 대책', '사회'), ('대책 시사', '정치'), ('부동산 대책', '사회'), ('부총리 전셋집', '사회'), ('전세 대책', '사회')]
[('부총리 전세', '사회'), ('대책 부총리', '경제'), ('과거 전세대책', '경제'), ('대책 임대주택', '경제'), ('부동산 정책', '정치')]
[('의원 김상조', '사회'), ('면서 전세', '경제'), ('청와대 정책', '경제'), ('가중 시장', '정치'), ('과도기 불안', '경제')]
[('금융 부동산', '경제'), ('이상 거주', '경제'), ('청약 제도로', '경제'), ('전세 월세', '경제'), ('임대차 시장', '경제')]
[('전월세 상한', '사회'), ('적용 월세', '경제'), ('사례 전월세', '경제'), ('취지 전월세', '경제'), ('전환율 주택임대차보호법', '경제')]
[('방역 경제', '정치'), ('임대차 조기', '경제'), ('정부 의지', '경제'), ('시장 안정', '경제'), ('문재인 대통령', '경제')]
[('일각 전세', '사회'), ('부총리 전세', '경제'), ('부총리 임대차', '경제'), ('전월세 전환율', '사회'), ('전세 불안', '경제')]
[('분양 물량', '경제'), ('전세 물량', '경제'), ('기준 분양', '경제'), ('앵커 집값', '경제'), ('아파트 전세', '경제')]
[('대통령비서실장', '경제'), ('대통령비서실장 조원', '사회'), ('만큼 청와대', '정치'), ('비서실 문재인', '경제'), ('비서실 사표', '사회')